In [1]:
import os
import json
import numpy as np
from lib import StyloAnalyzer

corpora = ['aurthur_conan_doyle', 'charles_dickens',
           'h_g_wells', 'jane_austen', 'jonathan_swift']
# corpora = ['aurthur_conan_doyle']


In [2]:
class StyloAnalyzer(StyloAnalyzer):
    def __init__(self, label: str, text: str) -> None:
        super().__init__(label, text)

In [3]:
analyzed_data = {}
for corpus in corpora:
    for entry in os.scandir(os.path.join('corpora_split', corpus)):
        if entry.is_file():
            with open(entry.path, 'r') as file:
                analyzed_data[corpus + '.' + entry.name] = StyloAnalyzer(corpus + '.' + entry.name, file.read())
        # break


In [4]:
result = {}
for data in analyzed_data.values():
    result[data.label] = data.vector
    
json_object = json.dumps(result)

with open(os.path.join('output', 'metrics.json'), 'w') as outfile:
    outfile.write(json_object)

In [5]:
result = {}
for data in analyzed_data.values():
    result[data.label] = data.json
    # for series in data.json:
    #     if series.startswith('av_words_per_'):
    #         continue
    #     result[data.label][series] = list(np.cumsum(data.json[series]))

json_object = json.dumps(result)

with open(os.path.join('output', 'plot_all.json'), 'w') as outfile:
    outfile.write(json_object)



In [6]:
result2 = {}
for label in result:
    author = label.split('.')[0]
    if author not in result2.keys():
        result2[author] = {}
    for series in result[label]:
        author_data = result2.get(author)
        if series not in author_data.keys():
            author_data[series] = []
        author_data[series].append(result[label][series])

result3 = {}
for author in result2:
    if author not in result3.keys():
        result3[author] = {}
    for series in result2[author]:
        if series.startswith('av_words_per_'):
            values = sorted(result2[author][series])
            Q1 = np.percentile(values, 25, interpolation='midpoint')
            Q2 = np.percentile(values, 50, interpolation='midpoint')
            Q3 = np.percentile(values, 75, interpolation='midpoint')
            IQR = Q3 - Q1
            Min = Q1 - 1.5*IQR
            Max = Q3 + 1.5*IQR
            result3[author][series] = [Min, Q1, Q2, Q3, Max]
            continue
        result3[author][series] = []
        data = list(zip(*result2[author][series]))
        for i in range(len(data)):
            values = sorted(data[i])
            Q1 = np.percentile(values, 25, interpolation='midpoint')
            Q2 = np.percentile(values, 50, interpolation='midpoint')
            Q3 = np.percentile(values, 75, interpolation='midpoint')
            IQR = Q3 - Q1
            Min = Q1 - 1.5*IQR
            Max = Q3 + 1.5*IQR
            result3[author][series].append([Min, Q1, Q2, Q3, Max])
        result3[author][series] = [list(np.cumsum(fd))
                                   for fd in list(zip(*result3[author][series]))]

json_object = json.dumps(result3)
with open(os.path.join('output', 'plot.json'), 'w') as outfile:
    outfile.write(json_object)